In [1]:
import os, sys
sys.path.append('/home/dcooper/git/Bridgeport/')
from interactive.interactive_utils import *
from utils.utils import write_FASTA
from Bridgeport.Bridgeport import Bridgeport
import py3Dmol
from datetime import datetime
from rdkit.Chem.Draw import rdDepictor

/home/dcooper/anaconda3/envs/prep/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


**Let's start with some basic information...**

**Protein Input**

- *input_pdb_dir*: (str) path to directory where the *input_pdb* can be found. 
- *input_pdb*: (str) name of input structure
- *chain*: (str) of chain to use from *input_pdb* to align

**Ligand Input** (if you are doing an analogue, fill in this information for the templace ligand in your experimental structure)

- *name*: (str) a name for your ligand
- *resname*: (str) resname of ligand found in *input_pdb*. If the ligand, is a peptide set to "False".
- *chainid*: (str) if ligand is a peptide, specify the letter code that denotes the ligand, if not set to "False".
- *smiles*: (str) smiles string for you ligand. 

In [2]:
# Set you working directory
input_dir = os.getcwd()

In [3]:
# Protein input
input_pdb_dir = os.path.join(input_dir, 'input_pdb') # Change as needed
input_pdb = 'AF-NT0-5-4GRV.pdb' # Change as needed
chain = 'A' # Change as needed

# # Ligand input
name = 'PD149163'
resname = False
chainid = 'B'
lig_smiles = "CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCN=C(N)N)NC(=O)[C@@H](N)CCCN=C(N)N)C(=O)N[C@@H](CC(C)C)C(=O)O"


In [4]:
# Build initial .json file and BP object
working_dir = os.getcwd()
json_dict = {'working_dir': working_dir}

# Update params
json_dict['Protein'] = {'input_pdb_dir': input_pdb_dir,
                        'input_pdb': input_pdb,
                        'chain': chain}


json_dict['Ligand'] = {'name': name,
                       'resname': resname,
                       'chainid': chainid,
                       'smiles': lig_smiles}

json_fn = 'interactive.json'
write_json(json_dict, json_fn)
start = datetime.now()
BP = Bridgeport(json_fn, verbose=False)

10/02/2025 10:37:29//Welcome to Bridgeport.
10/02/2025 10:37:29//Found input parameters.
10/02/2025 10:37:29//working_dir:
10/02/2025 10:37:29//working_dir: /home/dcooper/git/Bridgeport/interactive
10/02/2025 10:37:29//Protein:
10/02/2025 10:37:29//	input_pdb_dir: /home/dcooper/git/Bridgeport/interactive/input_pdb
10/02/2025 10:37:29//	input_pdb: AF-NT0-5-4GRV.pdb
10/02/2025 10:37:29//	chain: A
10/02/2025 10:37:29//Ligand:
10/02/2025 10:37:29//	name: PD149163
10/02/2025 10:37:29//	resname: False
10/02/2025 10:37:29//	chainid: B
10/02/2025 10:37:29//	smiles: CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCN=C(N)N)NC(=O)[C@@H](N)CCCN=C(N)N)C(=O)N[C@@H](CC(C)C)C(=O)O


1. **Align your protein to a reference structure**
- *reference_pdb*: (str) path to refence structure to align input structure to
- *reference_chain*: (List[str]) of chains of the *reference_pdb* to use in the alignment process

In [5]:
# Set input paths
reference_pdb = os.path.join(input_dir, 'OPM', '6z66_OPM.pdb') # Change as needed
reference_chain = ['A'] # Change as needed

# Update input params
BP.input_params['Environment'] = {'alignment_ref': reference_pdb,
                                  'reference_chain': reference_chain}

# Align
BP.align_to_reference()

# Visualizes aligned structures
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(BP.aligned_pdb, 'r').read(),'pdb')
view.addModel(open(reference_pdb, 'r').read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'blue'}})
view.setStyle({'model':1}, {'cartoon': {'color':'yellow'}})
view.zoomTo()
view.show()



10/02/2025 10:37:29//Found directory for aligned input structures: /home/dcooper/git/Bridgeport/interactive/aligned_input_pdb
10/02/2025 10:37:29//Found references structure /home/dcooper/git/Bridgeport/interactive/OPM/6z66_OPM.pdb and will align to chains ['A']
10/02/2025 10:37:29//Found input structure: /home/dcooper/git/Bridgeport/interactive/input_pdb/AF-NT0-5-4GRV.pdb
10/02/2025 10:37:29//Saved aligned structure to: /home/dcooper/git/Bridgeport/interactive/aligned_input_pdb/PD149163.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

2. **Seperate your ligand and protein for individual preparations**

In [6]:
# Separate
BP.separate_lig_prot()

10/02/2025 10:37:29//Found directory for protein structures: /home/dcooper/git/Bridgeport/interactive/proteins
10/02/2025 10:37:29//Found directory for ligand structures: /home/dcooper/git/Bridgeport/interactive/ligands
10/02/2025 10:37:29//Separated chain(s) A from input structure
10/02/2025 10:37:29//Separated ligand B from input structure


3. **Repair the protein**
   - *tails*: List of indices to parse the extra tails. EX: [30, 479]
   - *loops*: 2-D List of indices that specify lower and upper bounds of loops to optimize during refinement. Loop optimization can take a while, but if skipped, unbonded output structures will result.
   - *secondary_template*: Path to secondary .pdb to use as a reference to accurately model large portions that are missing in the input .pdb structure.
   - *engineered_resids*: List of resids that are known engineered mutations in the crystal pdb. Adding this argument may prevent sequence errors in the RepairProtein section of Bridgeport.
   - *receptor_gene*: (str) gene of the receptor
   - *sequence*: (str) sequence of the receptor you are trying to model

In [7]:
# Set input
tails = [48, 390] # Change as needed
loops = False # Change as needed
secondary_template = os.path.join(input_dir, 'input_pdb', 'AF-P30989-F1-model_v4.pdb') # Change as needed
engineered_resids = None # Change as needed
receptor_gene = 'NTSR1' # Change as needed
sequence = """
MRLNSSAPGTPGTPAADPFQRAQAGLEEALLAPGFGNASGNASERVLAAPSSELDVNTDIYSKVLVTAVYLALFVVGTVGNTVTAFTLARKKSLQSLQSTVHYHLGSLALSDLLTLLLAMPVELYNFIWVHHPWAFGDAGCRGYYFLRDACTYATALNVASLSVERYLAICHPFKAKTLMSRSRTKKFISAIWLASALLAVPMLFTMGEQNRSADGQHAGGLVCTPTIHTATVKVVIQVNTFMSFIFPMVVISVLNTIIANKLTVMVRQAAEQGQVCTVGGEHSTFSMAIEPGRVQALRHGVRVLRAVVIAFVVCWLPYHVRRLMFCYISDEQWTPFLYDFYHYFYMVTNALFYVSSTINPILYNLVSANFRHIFLATLACLCPVWRRRRKRPAFSRKADSVSSNHTLSSNATRETLY
"""

# Write FASTA
fasta_path = os.path.join(input_dir, 'fasta', f'{receptor_gene}.fasta')
write_FASTA(sequence, receptor_gene, fasta_path)

# Update params
BP.input_params['RepairProtein'] = {'fasta_path': fasta_path,
                                    'working_dir': os.path.join(os.getcwd(), 'modeller_intermediates'),
                                    'tails': tails, 
                                    'loops': loops,
                                    'secondary_template': secondary_template,
                                    'engineered_resids': engineered_resids}
# Repair
BP.repair_protein()

# Visualizes repaired protein
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(os.path.join(BP.prot_only_dir, BP.name + '.pdb'), 'r').read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'blue'}})
view.zoomTo()
view.show()

10/02/2025 10:37:29//Welcome to RepairProtein
10/02/2025 10:37:29//Protein to repair: /home/dcooper/git/Bridgeport/interactive/proteins/PD149163.pdb
10/02/2025 10:37:29//Template sequence: /home/dcooper/git/Bridgeport/interactive/fasta/NTSR1.fasta
10/02/2025 10:37:29//Modeller intermediates will be written to: /home/dcooper/git/Bridgeport/interactive/modeller_intermediates
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies

>> Summary of successfully produced models:
Filename                          molpdf
----------------------------------------
NTSR1.B99990001.pdb           8502.81348

10/02/2025 10:37:42//Moved protein from 1.4410298368852348 to 0.4148173840812806
10/02/2025 10:37:42//Protein Repaired. Output written to: /home/dcooper/git/Bridgeport/interactive/proteins/PD149163.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

4. **Build the environment**

- "membrane": If membrane should be specified choose "true", and make sure that "alignment_ref" argument is the appropriate OPM structure. Default is false.
- "pH": Specify the pH. Default is 7.0.
- "ion_strength": Specify the concentration of NaCl ions (in Molar). Default is 0.15 M.


In [8]:
# # Set input
# membrane = True # Change as needed
# pH = 7.2 # Change as needed
# ion_strength = 0.15 # Change as needed

# BP.input_params['Environment']['membrane'] = membrane
# BP.input_params['Environment']['pH'] = pH
# BP.input_params['Environment']['ion_strength'] = ion_strength

# BP.add_environment()

5. **Prepare the ligand**
- *small_molecule_params (bool)*: If true, treat ligand like a small molecule. Default is True.
- *sanitize (bool)*: If true, sanitize molecule with rdkit. Default is True. Only applicable if small_molecule_params is True. 
- *removeHs (bool)*: If true, remove any hydrogens that may be present. Default is True. Only applicable if small_molecule_params is True.
- *proximityBonding* (bool): If true, use rdkit's 'proximityBonding' method to load rdkit molecule. 
- *pH (float)*: pH to protonate a peptide ligand. Default is 7.0.
- *nstd_resids (List[int])*: List of nonstandard resids to conserve from input structure. 
- *neutral_Cterm (bool)*: If true, neutralize the C-terminus of a peptide ligand. Only applicable is small_molecule_params is False


In [9]:
# Ligand Input for PD149163
sequence = 'KPWG'
nstd_resids = None # Change as needed
pH = 7.2 # Change as needed
chain = 'P'
removeHs = True

# Update params
BP.input_params['Ligand']['nstd_resids'] = nstd_resids
BP.input_params['Ligand']['pH'] = pH
BP.input_params['Ligand']['removeHs'] = removeHs
BP.input_params['Ligand']['chain'] = chain
BP.input_params['Ligand']['sequence'] = sequence
BP.input_params['Ligand']['MutatedPeptide'] = {}
BP.input_params['Ligand']['MutatedPeptide']['Mutations'] = []

BP.input_params['Ligand']['MutatedPeptide']['Mutations'].append({})
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['mutation_resid'] = 1
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['mutation_resname'] = 'DHL'
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['mutation_smiles'] = 'NC(=O)[C@H](CCCC[NH3+])[NH2+]C[C@@H]([NH3+])CCCC[NH3+]'
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['remove_atoms'] = ['N1', 'H1', 'H2']
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['change_atoms'] = [{'N1': 'protein-N', 'H1': 'protein-H'}]
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['bonds_to_add'] = [[[1, 'C1'], [2, 'N']]]
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['external_bonds'] = ['C1']

BP.input_params['Ligand']['MutatedPeptide']['Mutations'].append({})
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['mutation_resid'] = 4
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['mutation_resname'] = 'MEV'
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['mutation_smiles'] = 'CCOC(=O)[C@H](CC(C)C)NC(=O)[C@@H](NC=O)C(C)(C)C'
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['remove_atoms'] = ['C11', 'O4', 'H19']
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['change_atoms'] = [{'C11': 'protein-C', 'O4': 'protein-O'}]
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['bonds_to_add'] = [[[3, 'C'], [4, 'N2']]]
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['external_bonds'] = ['N2']

# BP.input_params['Ligand']['MutatedPeptide']['Mutations'].append({})
# BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['mutation_resid'] = 5
# BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['mutation_resname'] = 'LEE'
# BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['mutation_smiles'] = 'CCOC(=O)[C@H](CC(C)C)NC=O'
# BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['remove_atoms'] = ['H17', 'C9', 'O3']
# BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['change_atoms'] = [{'C9': 'protein-C', 'O3': 'protein-O'}, {'N1': 'protein-N', 'H16': 'protein-H'}]
# BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['bonds_to_add'] = [[[4, 'C7'], [5, 'N1']]]
# BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['external_bonds'] = ['N1']

# Prepare
BP.ligand_prep()

# Visualizes ligand sdf and pdb
view = py3Dmol.view()
view.setBackgroundColor('white')
if hasattr(BP, 'lig_sdf'):
    view.addModel(open(BP.lig_sdf, 'r').read(),'sdf')
else:
    view.addModel(open(BP.lig_pdb, 'r').read(),'pdb')
view.setStyle({'model':0}, {'stick': {'colorscheme':'cyanCarbon'}})
view.zoomTo()
view.show()


10/02/2025 10:37:42//Found peptide ligand with resname: P
10/02/2025 10:37:42//Welcome to RepairProtein
10/02/2025 10:37:42//Protein to repair: /home/dcooper/git/Bridgeport/interactive/ligands/DHL1_reference.pdb
10/02/2025 10:37:42//Template sequence: /home/dcooper/git/Bridgeport/interactive/lig.fasta
10/02/2025 10:37:42//Modeller intermediates will be written to: /home/dcooper/git/Bridgeport/interactive/modeller
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies

>> Summary of successfully produced models:
Filename                          molpdf
----------------------------------------
lig.B99990001.pdb               12.86549

10/02/2025 10:37:42//Moved protein from 3.1727148220202435 to 0.9169029599534914
10/02/2025 10:37:42//Protein Repaired. Output written to: /home/dcooper/git/Bridgeport/interactive/ligands/DHL1_reference.pdb
10/02

INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: /home/dcooper/git/Bridgeport/interactive/ligands/DHL1_reference.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     285 UNITARY VALUES FOR THE UNIT CELL AUTOMATICALLY SET>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 4 residues and 35 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:This biomolecu

10/02/2025 10:37:42//Saved conformer to /home/dcooper/git/Bridgeport/interactive/ligands/DHL1_residue_conformers/DHL1_residue_0.pdb
10/02/2025 10:37:42//Saved first conformer to /home/dcooper/git/Bridgeport/interactive/ligands/DHL1_residue.pdb
10/02/2025 10:37:42//Saved prepared ligand to /home/dcooper/git/Bridgeport/interactive/ligands/DHL1_residue.pdb /home/dcooper/git/Bridgeport/interactive/ligands/DHL1_residue.sdf
Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to amber; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for pdb File --
   Status: pass
Info: Total number of electrons: 144; net charge: 4

Running: /home/dcooper/anaconda3/envs/prep/bin/sqm -O -i sqm.in -o sqm.out

-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/prep to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/lib to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/

[10:38:47] WARNING: More than one matching pattern found - picking one



Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to amber; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for pdb File --
   Status: pass
Info: Total number of electrons: 164; net charge: 0

Running: /home/dcooper/anaconda3/envs/prep/bin/sqm -O -i sqm.in -o sqm.out

-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/prep to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/lib to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/parm to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/cmd to search path.
-f: Source /home/dcooper/git/Bridgeport/interactive/ligands/MEV4_residue.tleap.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: /home/dcooper/git/Bridgeport/interactive/ligands/MEV4_residue.tleap
----- Source: /home/dcooper/anaconda3/envs/prep/dat/leap/cmd/leaprc.gaff
----- Source of /home/dcooper/anaconda3/envs/p

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

6. **Build the forcefields**


In [10]:
BP.env_pdb = '/home/dcooper/git/Bridgeport/interactive/proteins/PD149163_env.pdb'
BP.generate_systems()

10/02/2025 10:39:16//Building parameters for PD149163
10/02/2025 10:39:18//Protein parameters built.
10/02/2025 10:39:18//Ligand parameters built.
10/02/2025 10:39:18//System parameters built.
10/02/2025 10:39:19//Initial structure potential energy: 1.1316834394709586e+16


**Save your input_parameters to a .json file**
- *json_fn*: str path to .json file where you would like to save your input parameters

In [11]:
# Change ligand smiles back
# BP.input_params['Ligand']['smiles'] = lig_smiles

# # Set .json location
# json_fn = os.path.join(input_dir, 'json', f'{name}.json')
# write_json(BP.input_params, json_fn)

# print('WROTE:', json_fn)
# print('TOOK:', datetime.now() - start)

In [12]:
# from openmm.app import *

# pdb = PDBFile('/home/dcooper/projects/GIP/ligands/AIB2_reference.pdb')
# top = pdb.getTopology()
# for a in top.atoms():
#     print(a.index)

### Minimize

In [13]:
from Minimizer.Minimizer import Minimizer
os.makedirs('test', exist_ok=True)
row = Minimizer(BP.final_xml, BP.final_pdb, 'test')
row._minimize(BP.final_pdb, os.path.join('test', 'minimized.pdb'))

Unpacking /home/dcooper/git/Bridgeport/interactive/systems/PD149163.xml, /home/dcooper/git/Bridgeport/interactive/systems/PD149163.pdb
Original state has energy 1.131683421166923e+16 kJ/mol  with maximum force 3719550745.53 kJ/(mol nm)
Minimized state has energy -1246939.75 kJ/mol  with maximum force 2978.66 kJ/(mol nm)
Minimization completed in 0:01:06.622445
Wrote: test/minimized.pdb
Wrote: /home/dcooper/git/Bridgeport/interactive/test/minimized.xml


('/home/dcooper/git/Bridgeport/interactive/test/minimized.xml',
 'test/minimized.pdb')